# Draftnet bot development
This notebook tests the performance of a series of bots from draftsimtools.

In [74]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from operator import itemgetter
from copy import deepcopy
import json
import torch
from sklearn.model_selection import train_test_split

import draftsimtools as ds

In [4]:
# Pytorch device.
device = torch.device("cpu")

### Load M19 drafts. 
Data folder is downloadable from draft-data-files on slack

In [7]:
raw_drafts = ds.load_drafts("../data/subset3000/test.csv")
# raw_drafts = ds.load_drafts("../../data/m19_2.csv") # Previous import.

This cell isn't runnable without AllSets.json

In [8]:
# Here other folks load card lists, but I grab them from json instead
# m19_set = ds.create_set("data/m19_rating.tsv", "data/m19_land_rating.tsv")

with open('../data/AllSets.json', 'r',encoding='utf-8') as json_data:
    mtgJSON = json.load(json_data)
jsonSubset = mtgJSON['M19']['cards']
thisSet = {card['name'] : card for card in jsonSubset}

In [9]:
# Another (fancier) way to create a list of names + lots of other useful stuff
nameList = pd.DataFrame.from_dict(thisSet, orient='index') #, columns=['colors','rarity','type','convertedManaCost'])
nameList['Name'] = nameList.index                 # We need names as a column, not an index
nameList['index'] = range(len(nameList))
nameList = nameList.set_index('index')     # And we need a normal numerical index
nameList[1:5]

# Process names, then handle weird card names (those with commas)
nameList['Name'] = nameList.Name.str.replace(' ', '_')

# This utility method searches for "Name" column in nameList that have commas
nameList, raw_drafts = ds.fix_commas(nameList, raw_drafts) # Returns a tuple, as it updates both

# Prints namelist
#nameList[1:5]

Load sets normally instead.

In [10]:
m19_set = ds.create_set("../data/m19_rating.tsv", "../data/m19_land_rating.tsv")

In [11]:
m19_set = m19_set.sort_values("Name").reset_index(drop=True)

In [12]:
# Process the drafts, deconstructing packs (hands) at every turn of every draft
drafts = ds.process_drafts(raw_drafts)

Processing draft: 0.


In [13]:
# Splits into (toy) training and test sets. NOTE: For real training, use all drafts.
subset_drafts = drafts[:20] # We are only looking at 1000 with this import.
#subset_drafts = drafts[5000:5500]
#train, test = train_test_split(subset_drafts, test_size = 0.4)

In [14]:
# This should simply add basic lands. 

# Make sure all cards are listed in the nameList; update if necessary
for iDraft in range(len(subset_drafts)):
    draft = subset_drafts[iDraft]
    for pack in draft:     
        for cardName in pack:
            try:
                pos = nameList[nameList.Name==cardName].index[0]
            except:
                print("---Unrecognized card: ",cardName) # All unrecognized cards here seem to be foil lands
                #  	colors 	rarity 	type 	convertedManaCost 	Name
                nameList = nameList.append({'colors':[],'rarity':'weird','type':'weird',
                                            'convertedManaCost':0,'Name':cardName},ignore_index=True)

---Unrecognized card:  Island_2
---Unrecognized card:  Plains_4
---Unrecognized card:  Plains_3
---Unrecognized card:  Swamp_4
---Unrecognized card:  Forest_3
---Unrecognized card:  Forest_2
---Unrecognized card:  Swamp_2
---Unrecognized card:  Island_3
---Unrecognized card:  Mountain_4
---Unrecognized card:  Mountain_3
---Unrecognized card:  Island_4
---Unrecognized card:  Swamp_1
---Unrecognized card:  Mountain_2
---Unrecognized card:  Swamp_3
---Unrecognized card:  Forest_1
---Unrecognized card:  Forest_4
---Unrecognized card:  Island_1
---Unrecognized card:  Plains_2
---Unrecognized card:  Plains_1
---Unrecognized card:  Mountain_1


In [15]:
print(len(nameList))

320


In [16]:
# Splits data into training and testing sets.
# This doesn't matter for some bots, but for others it does,
# so we want to make sure we evaluate on the same testing
# set for every bot. 
train, test = train_test_split(subset_drafts, test_size = 0.4)

In [17]:
# Tests the random and the raredrafting bots against each other
bot1 = ds.RandomBot()
# bot2 = ds.RaredraftBot(nameList) # These bots are not runnable without allsets json.
# bot3 = ds.ClassicBot(nameList)
bot4 = ds.RandomBot()

In [18]:
tester = ds.BotTester(subset_drafts)

In [19]:
#tester.evaluate_bots([bot1, bot2, bot3], ["RandomBot", "RaredraftBot", "ClassicBot"])
tester.evaluate_bots([bot1, bot4], ["RandomBot", "MiscBot"])
tester.report_evaluations()
tester.write_evaluations()

draft_num    10.500000
pick_num     23.000000
RandomBot     0.217778
MiscBot       0.227778
dtype: float64
Wrote correct to: output_files/exact_correct.tsv
Wrote fuzzy_correct to: output_files/fuzzy_correct.tsv
Wrote rank_error to: output_files/rank_error.tsv
Wrote card_acc to: output_files/card_accuracies.tsv


# Define draftnet (required for torch import)

In [22]:
# This cell now included in draftsimtools (without GPU support).
def create_le(cardnames):
    """Create label encoder for cardnames."""
    le = preprocessing.LabelEncoder()
    le.fit(cardnames)
    return le

def draft_to_matrix(cur_draft, le, pack_size=15):
    """Transform draft from cardname list to one hot encoding."""
    pick_list = [np.append(le.transform(cur_draft[i]), (pack_size-len(x))*[0]) \
                 for i, x in enumerate(cur_draft)]
    pick_matrix = np.int16(pick_list, device=device)
    return pick_matrix

def drafts_to_tensor(drafts, le, pack_size=15):
    """Create tensor of shape (num_drafts, 45, 15)."""
    pick_tensor_list = [draft_to_matrix(d, le) for d in drafts]
    pick_tensor = np.int16(pick_tensor_list, device=device)
    return pick_tensor

#Drafts dataset class.
class DraftDataset(Dataset):
    """Defines a draft dataset in PyTorch."""
    
    def __init__(self, drafts_tensor, le):
        """Initialization.
        """
        self.drafts_tensor = drafts_tensor
        
#Torch imports.        
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data.dataset import Dataset

#Implement NN.
class DraftNet(nn.Module):
    
    def __init__(self, set_tensor):
        """Placeholder NN. Currently does nothing.
        
        param ss: number of cards in set
        param set_tensor: Mxss set tensor describing the set
        """
        super(DraftNet, self).__init__()
        
        # Load set tensor.
        self.set_tensor = set_tensor
        self.set_tensor_tranpose = torch.transpose(set_tensor, 0, 1)
        self.M, self.ss = self.set_tensor.shape
        self.half_ss = self.ss / 2
        
        # Specify layer sizes. 
        size_in = self.ss + self.M
        #size_in = self.ss
        size1 = self.ss
        size2 = self.ss
        size3 = self.ss
        size4 = self.ss
        size5 = self.ss
        size6 = self.ss
        size7 = self.ss
        size8 = self.ss
        
        self.ns = 0.01
        
        self.bn = nn.BatchNorm1d(self.ss + self.M)
        
        self.linear1 = torch.nn.Linear(size_in, size1)
        self.bn1 = nn.BatchNorm1d(size1)
        self.relu1 = torch.nn.LeakyReLU(negative_slope = self.ns)
        self.dropout1 = nn.Dropout(0.5)
        
        self.linear2 = torch.nn.Linear(size1, size2)
        self.bn2 = nn.BatchNorm1d(size2)
        self.relu2 = torch.nn.LeakyReLU(negative_slope = self.ns)
        self.dropout2 = nn.Dropout(0.5)
        
        self.linear3 = torch.nn.Linear(size2, size3)
        self.bn3 = nn.BatchNorm1d(size3)
        self.relu3 = torch.nn.LeakyReLU(negative_slope = self.ns)
        self.dropout3 = nn.Dropout(0.5)
        
        self.linear4 = torch.nn.Linear(size3, size4)
        self.relu4 = torch.nn.LeakyReLU(negative_slope = self.ns)
        self.dropout4 = nn.Dropout(0.5)
        
        self.linear5 = torch.nn.Linear(size3, size5)
        self.relu5 = torch.nn.LeakyReLU(negative_slope = self.ns)
        self.dropout5 = nn.Dropout(0.5)
        
        self.linear6 = torch.nn.Linear(size3, size6)
        self.relu6 = torch.nn.LeakyReLU(negative_slope = self.ns)
        self.dropout6 = nn.Dropout(0.5)
        
        self.linear7 = torch.nn.Linear(size3, size7)
        self.relu7 = torch.nn.LeakyReLU(negative_slope = self.ns)
        self.dropout7 = nn.Dropout(0.5)
        
        self.linear8 = torch.nn.Linear(size3, size8)
        self.relu8 = torch.nn.LeakyReLU(negative_slope = self.ns)
        
        
        #self.sm = torch.nn.Softmax()
                
    def forward(self, x):
        
        collection = x[:, :self.ss]
        
        #collection = self.bn(collection)
        
        pack = x[:, self.ss:]
        
        # Get features from set tensor. 
        features = torch.mm(collection, self.set_tensor_tranpose)
        collection_and_features = torch.cat((collection, features), 1)
        
        collection_and_features = self.bn(collection_and_features)
        
        #y = self.linear1(collection_and_features)
        y = self.linear1(collection_and_features)
        y = self.bn1(y)
        y = self.relu1(y)
        y = self.dropout1(y)
        
        y = self.linear2(y)
        y = self.bn2(y)
        y = self.relu2(y)
        y = self.dropout2(y)
        
        y = self.linear3(y)
        y = self.bn3(y)
        y = self.relu3(y)
        y = self.dropout3(y)

        y = self.linear4(y)
        #y = self.relu4(y)
        #y = self.dropout4(y)
        
        #y = self.linear5(y)
        #y = self.relu5(y)
        #y = self.dropout5(y)
        
        #y = self.linear6(y)
        #y = self.relu6(y)
        #y = self.dropout6(y)
        
        #y = self.linear7(y)
        #y = self.relu7(y)
        #y = self.dropout7(y)
        
        #y = self.linear8(y)
        #y = self.relu8(y)
        
        y = y * pack # Enforce cards in pack only.
        
        return y
        self.le = le
        self.cards_in_set = len(self.le.classes_)
        self.pack_size = int(self.drafts_tensor.shape[1]/3)
        self.draft_size = self.pack_size*3
        
    def __getitem__(self, index):
        """Return a training example.
        """
        #Grab information on current draft.
        pick_num = index % self.draft_size #0-self.pack_size*3-1
        draft_num = int((index - pick_num)/self.draft_size)
        
        #Generate.
        x = self.create_new_x(pick_num, draft_num)
        y = self.create_new_y(pick_num, draft_num)
        return x, y
    
    def create_new_x(self, pick_num, draft_num):
        """Generate x, input, as a row vector.
        0:n     : collection vector
                  x[i]=n -> collection has n copies of card i
        n:2n    : pack vector
                  0 -> card not in pack
                  1 -> card in pack
        Efficiency optimization possible. Iterative adds to numpy array.
        """
        #Initialize collection / cards in pack vector.
        x = np.zeros([self.cards_in_set * 2], dtype = "int16")
        
        #Fill in collection vector excluding current pick (first half).
        for n in self.drafts_tensor[draft_num, :pick_num, 0]:
            x[n] += 1
            
        #Fill in pack vector.
        cards_in_pack =  self.pack_size - pick_num%self.pack_size #Cards in current pack.
        for n in self.drafts_tensor[draft_num, pick_num, :cards_in_pack]:
            x[n + self.cards_in_set] = 1
            
        #Convert to Torch tensor.
        x = torch.Tensor(x)
        return x
    
    def create_new_y(self, pick_num, draft_num, not_in_pack=0.5):
        """Generate y, a target pick row vector.
        Picked card is assigned a value of 1.
        Other cards are assigned a value of 0.
        """
        #Initialize target vector.
        #y = np.array([0] * self.cards_in_set)
        y = np.zeros([self.cards_in_set], dtype = "int16")
            
        #Add picked card.
        y[self.drafts_tensor[draft_num, pick_num, 0]] = 1
        #y = torch.Tensor(y, dtype=torch.int64) # Needed as target.
        y = torch.tensor(y, dtype=torch.int64, device=device) # Needed as target.
        return y
    
    def __len__(self):
        return len(self.drafts_tensor) * self.draft_size

def load_dataset(rating_path1, rating_path2, drafts_path):
    """Create drafts tensor from drafts and set files."""
    # Load the set. inputs
    cur_set = ds.create_set(rating_path1, rating_path2)
    raw_drafts = ds.load_drafts(drafts_path)
    
    # Fix commas. 
    cur_set, raw_drafts = ds.fix_commas(cur_set, raw_drafts)
    
    # Process drafts. 
    drafts = ds.process_drafts(raw_drafts)
    
    # Drop empty elements at end, if present. 
    while len(drafts[-1]) == 0:
        drafts = drafts[:-1]
    
    # Create a label encoder.
    le = create_le(cur_set["Name"].values)
    
    # Create drafts tensor. 
    drafts_tensor = drafts_to_tensor(drafts, le)
    
    # Create a dataset.
    cur_dataset = DraftDataset(drafts_tensor, le)
    
    # Get the tensor
    return cur_dataset, drafts_tensor, cur_set, le

# Load network from disk

In [26]:
test_net = torch.load("./bots_data/draftnet_oct_17_2019_633_cpu.pt")
test_net.eval()

DraftNet(
  (bn): BatchNorm1d(306, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear1): Linear(in_features=306, out_features=285, bias=True)
  (bn1): BatchNorm1d(285, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): LeakyReLU(negative_slope=0.01)
  (dropout1): Dropout(p=0.5)
  (linear2): Linear(in_features=285, out_features=285, bias=True)
  (bn2): BatchNorm1d(285, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): LeakyReLU(negative_slope=0.01)
  (dropout2): Dropout(p=0.5)
  (linear3): Linear(in_features=285, out_features=285, bias=True)
  (bn3): BatchNorm1d(285, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): LeakyReLU(negative_slope=0.01)
  (dropout3): Dropout(p=0.5)
  (linear4): Linear(in_features=285, out_features=285, bias=True)
  (relu4): LeakyReLU(negative_slope=0.01)
  (dropout4): Dropout(p=0.5)
  (linear5): Linear(in_features=285, out_features=285, bias=True)
  (relu5): LeakyR

In [27]:
# Define rating file paths. 
rating_path1 = "../data/m19_rating.tsv"
rating_path2 = "../data/m19_land_rating.tsv"
drafts_path = "../data/subset3000/train.csv"

# Load data. 
train_data, train_tensor, m19_set, le = ds.load_dataset(rating_path1, rating_path2, drafts_path)

Processing draft: 0.


In [30]:
# 1. Collection, pack -> x

# Note: draft_frame = [collection, pack]


# 2. Draftnet on x
# 3. Return ranked output

In [33]:
# Draft frame only exists internall in the bot_tester class.
# Pragmatically speaking, make up a fake collection. 

In [32]:
def collection_pack_to_x(collection, pack, le):
    """Generate x, input, as a row vector.
    0:n     : collection vector
              x[i]=n -> collection has n copies of card i
    n:2n    : pack vector
              0 -> card not in pack
              1 -> card in pack
              
    :param collection: cardnames in collection list[string]
    :param pack: cardnames in pack list[string]
    :param le: label encoder
    
    :return: x vector
    """
    
    #Initialize collection / cards in pack vector.
    cards_in_set = len(le.classes_)    
    x = np.zeros([cards_in_set * 2], dtype = "int16")

    #Fill in collection vector.
    collection_indices = le.transform(collection)
    for ci in collection_indices:
        x[ci] += 1

    #Fill in pack vector.
    pack_indices = le.transform(pack)
    for pi in pack_indices:
        x[pi + cards_in_set] += 1

    #Convert to Torch tensor.
    x = torch.Tensor(x).reshape(1, -1) # Include batch dimension.
    return x

# Run on an example pack.

In [78]:
# Create demo collection. 
demo_collection = tester.drafts[0][0]
demo_pack = tester.drafts[0][1]
demo_x = collection_pack_to_x(demo_collection, demo_pack, le)

# Return the result. 
result = test_net(demo_x)

# Compute the ranked result.
pack_tuples = [(i, float(v.detach().numpy())) for i, v in enumerate(result[0, :]) if v > 0]
pack_tuples.sort(key = lambda t: t[1], reverse=True)
ranked_card_names = [le.inverse_transform(t[0]) for t in pack_tuples]

# Display the result. 
display(ranked_card_names)

['Knightly_Valor',
 'Exclusion_Mage',
 'Dwindle',
 'Rise_from_the_Grave',
 'Daggerback_Basilisk',
 "Knight's_Pledge",
 'Skeleton_Archer',
 'Epicure_of_Blood',
 'Anticipate',
 'Cancel',
 'Hired_Blade',
 'Hostile_Minotaur',
 'Crash_Through',
 'Plains_4']

## To do: implement in bot class